In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
REPO_NAME = 'sewer-nfl'
import sys, os
cwd = str(os.getcwd())
repo_dir = cwd[:cwd.find(REPO_NAME)+len(REPO_NAME)]
sys.path.insert(0,repo_dir)

In [17]:
from models.pe1.pipeline import model_df

2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


In [28]:
from models.pe1.pipeline import pbp_api_data

In [81]:
import numpy as np
import pandas as pd
final_model_data = pbp_api_data[['game_id','season','week','home_team','away_team','home_score',
                                 'away_score']]\
    .drop_duplicates().merge(
        model_df,
        how = 'left'
    )
final_model_data['home_cover'] = final_model_data.apply(lambda x: 1 if x['home_score'] - \
                                                                  x['away_score'] < \
                                                                  x['spread_line'] else 0,
                                                         axis = 1)
final_model_data = final_model_data.apply(pd.to_numeric, errors='coerce', axis=1)


LABEL = 'home_cover'
PREDICTORS = [
    'spread_line',
    'def_elo_QB_pass_x', 'off_elo_QB_pass_x', 'def_elo_RB_rec_x',
    'def_elo_TE_rec_x', 'def_elo_WR_rec_x', 'off_elo_RB_rec_x',
    'off_elo_TE_rec_x', 'off_elo_WR_rec_x', 'def_elo_QB_rush_x',
    'def_elo_RB_rush_x', 'def_elo_TE_rush_x', 'def_elo_WR_rush_x',
    'off_elo_QB_rush_x', 'off_elo_RB_rush_x', 'off_elo_TE_rush_x',
    'off_elo_WR_rush_x', 'posteam_y', 'defteam_y', 'def_elo_QB_pass_y',
    'off_elo_QB_pass_y', 'def_elo_RB_rec_y', 'def_elo_TE_rec_y',
    'def_elo_WR_rec_y', 'off_elo_RB_rec_y', 'off_elo_TE_rec_y',
    'off_elo_WR_rec_y', 'def_elo_QB_rush_y', 'def_elo_RB_rush_y',
    'def_elo_TE_rush_y', 'def_elo_WR_rush_y', 'off_elo_QB_rush_y',
    'off_elo_RB_rush_y', 'off_elo_TE_rush_y', 'off_elo_WR_rush_y'
]

In [82]:
X, y = final_model_data[PREDICTORS], final_model_data[[LABEL]]

In [83]:
from sklearn.preprocessing import OrdinalEncoder
import xgboost as xgb
y_transform = OrdinalEncoder().fit_transform(y)

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_transform, random_state=1, stratify=y_transform)

In [86]:
# Create classification matrices
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [88]:
params = {"objective": "multi:softprob", "tree_method": "gpu_hist", "num_class": 2}
n = 1000

results = xgb.cv(
   params, dtrain_clf,
   num_boost_round=n,
   nfold=5,
   metrics=["mlogloss", "auc", "merror"],
)

In [90]:
results

,train-mlogloss-mean,train-mlogloss-std,train-auc-mean,train-auc-std,train-merror-mean,train-merror-std,test-mlogloss-mean,test-mlogloss-std,test-auc-mean,test-auc-std,test-merror-mean,test-merror-std
0,0.647864,0.006119,0.741491,0.020628,0.327264,0.013854,0.694819,0.003557,0.526692,0.015808,0.489628,0.013847
1,0.612232,0.007901,0.813158,0.013959,0.269686,0.018041,0.697931,0.007712,0.535686,0.029060,0.472006,0.022917
2,0.585664,0.009714,0.853947,0.012471,0.232866,0.013423,0.699330,0.006749,0.542499,0.026951,0.466898,0.029338
3,0.563522,0.011435,0.876073,0.012771,0.210148,0.015067,0.704650,0.010679,0.539194,0.031471,0.469255,0.024630
4,0.541899,0.014904,0.897616,0.015057,0.186745,0.019680,0.708231,0.014761,0.537788,0.036691,0.471608,0.025503
...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.001544,0.000008,1.000000,0.000000,0.000000,0.000000,1.394608,0.033158,0.514343,0.005030,0.487662,0.010143
996,0.001543,0.000008,1.000000,0.000000,0.000000,0.000000,1.394504,0.032907,0.514272,0.004934,0.486878,0.009316
997,0.001542,0.000009,1.000000,0.000000,0.000000,0.000000,1.394625,0.033060,0.514365,0.004975,0.487663,0.009397
998,0.001541,0.000009,1.000000,0.000000,0.000000,0.000000,1.394392,0.033092,0.514322,0.004961,0.487662,0.009275
